<a href="https://colab.research.google.com/github/nghuixin/USC_BA_estimator/blob/main/IrimiaBrainAge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd  "/content/drive/MyDrive/"

/content/drive/MyDrive


In [ ]:
#!git clone https://github.com/irimia-laboratory/USC_BA_estimator.git # change folder name from USC_BA_estimator to test_brain_age
%cd /content/drive/MyDrive/Colab\ Notebooks/test_brain_age
!pip install -r requirements.txt
#!pip install pyfreesurfer

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/test_brain_age

/content/drive/MyDrive/Colab Notebooks/test_brain_age


In [23]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
# Path to our Jupyter notebook.
sys.path.append(os.path.join("/content/drive/MyDrive/Colab Notebooks/test_brain_age", "function"))


print(os.getcwd())
import nibabel.freesurfer.mghformat as mgh
from scipy.ndimage import zoom
import datetime
import pandas as pd
from preprocess import processmgz
import savepath_updated
import repadding
from pandas import read_csv
import tensorflow as tf
from tensorflow import keras
import modelcnn
import saliencymaps
import multiprocessing as mp

/content/drive/MyDrive/Colab Notebooks/test_brain_age


In [24]:
def mainfunction(dir_test_brain_age, path_to_model_weights, path_to_brain_mri):
    # we supposed the brain.mgz files are saved under this format: "/path_to_brain_mri/subject_id/brain.mgz".
    path_of_mgz = os.listdir(path_to_brain_mri)
    brains_tmp = []
    subj_id = []
    for mgz_id in path_of_mgz:
      print(mgz_id)
      if os.path.isdir(path_to_brain_mri+"/"+mgz_id) and os.path.exists(path_to_brain_mri+"/"+mgz_id+"/brain.mgz"):
        subj_id.append(mgz_id)
        print(subj_id)
        brains_tmp.append(zoom(mgh.load(path_to_brain_mri+"/"+mgz_id+"/brain.mgz").get_fdata(), (0.5, 0.5, 0.5)))
        print(brains_temp)
        print(mgh)

    brains, coordinates = processmgz(brains_tmp)
    (
        brain_save_path,
        coordinates_save_path,
        coordinates_save_path_csv,
    ) = savepath_updated.filename_brainnpy(dir_test_brain_age)
     # save the cropped brains
    np.save(brain_save_path, brains)

    # save information for repadding.
    np.save(coordinates_save_path, coordinates)
    df_coordinates = pd.DataFrame(coordinates)
    df_coordinates.columns = ["x", "y", "z"]
    df_coordinates["subj_id"] = subj_id
    df_coordinates.to_csv(coordinates_save_path_csv)

    # Build model.
    model = modelcnn.get_model(width=82, height=86, depth=100)

    # Load best weights.
    model.load_weights(path_to_model_weights)

    # make brain age predictions
    predictions = model.predict(brains) + 22

    # save the prediction results. Of note, the input subject ids should be saved as csv.

    BA_save_path_csv = savepath_updated.filename_pred(dir_test_brain_age)
    df_BA = pd.DataFrame(predictions)
    df_BA.columns = ["BA"]
    df_BA["subj_id"] = subj_id
    df_BA.to_csv(BA_save_path_csv)

    # Generate saliency mapes
    saliency_map_list = saliencymaps.smap(model, brains)

    # Repadding the saliency maps
    coordinates = np.load(coordinates_save_path)
    repadded_saliency_maps = repadding.repadding(saliency_map_list, coordinates)

    # Save the saliency maps
    smap_path = savepath_updated.filename_smap(dir_test_brain_age)
    np.save(smap_path, repadded_saliency_maps)
    return df_BA, repadded_saliency_maps

In [25]:
!pwd

/content/drive/MyDrive/Colab Notebooks/test_brain_age


In [26]:
# path to the folder
dir_test_brain_age = "/content/drive/MyDrive/Colab Notebooks/test_brain_age"

# path to the h5 file
path_to_model_weights = os.path.join(dir_test_brain_age, "function/model/saved-model-317.h5")
print(path_to_model_weights)

# path to the brain.mgz files
path_to_brain_mri = os.path.join(dir_test_brain_age, "brain_mri")
print(path_to_brain_mri)

# the mainfunction will automatically save and output the predicted BAs and saliency maps.
predicted_BA, repadded_saliency_maps = mainfunction(dir_test_brain_age, path_to_model_weights, path_to_brain_mri)

/content/drive/MyDrive/Colab Notebooks/test_brain_age/function/model/saved-model-317.h5
/content/drive/MyDrive/Colab Notebooks/test_brain_age/brain_mri
s223
s100
run processmgz
(0,)


AxisError: axis 4 is out of bounds for array of dimension 2